<a href="https://colab.research.google.com/github/jmestanza/natural-language-processing-practice/blob/main/desafios/Desafio_2/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crear sus propios vectores con Gensim
basado en lo visto en clase con otro
dataset.
Probar términos de interés y explicar
similitudes en el espacio de embeddings.
Intentar plantear y probar tests de
analogías. Graficar los embeddings
resultantes.
Sacar conclusiones.

Algunos de los sitios de textos sugeridos fueron:
- [Project Gutenberg](https://www.gutenberg.org/browse/scores/top)
- [Textos.info](https://www.textos.info/)

Se eligió analizar los textos pertenecientes a [The Project Gutenberg eBook of The Adventures of Sherlock Holmes](https://www.gutenberg.org/cache/epub/1661/pg1661.txt)
